In [219]:
import pandas as pd
import numpy as np 
from bs4 import BeautifulSoup
import requests
import datetime 
import time
import pytz
import os

In [220]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
last_cats = cat_history_df[last_mask]['name'].unique()
this_cats = cat_history_df[this_mask]['name'].unique()

diff = [x for x in this_cats if x not in last_cats]
print(diff) 

### Testing Foster List Data Collection

In [62]:
import sys
sys.path.insert(1, '../src')
from cafe_cats import *
from foster_cats import *
import re
from utilities import *

### Create old and New Files

In [166]:
df = getFosterInfo()

In [108]:
#make copy for edits
new_df = df.copy()

#changes to fields in existing rows
new_df.loc[3, 'cat_summary'] = 'test test test'
new_df.loc[7, 'cat_description'] = 'test test test'

#cat gets reserved
new_df.loc[9, 'cat_name'] = new_df.loc[9, 'cat_name'] + ' (reserved)'

#cat removed from foster list 
new_df.drop(12, inplace=True)

#cat added to list 
new_row = {
    'cat_id':['test-cat'],
    'cat_name':['test cat'],
    'cat_summary':['test'],
    'cat_timing':['test'],
    'cat_description':['test'],
    'cat_image':['test'],
    'cat_link':['test'],
    'import_at':[datetime.fromtimestamp(int(time.time()), tz=pytz.utc)]
}
new_row_df = pd.DataFrame(new_row)
new_df = pd.concat([new_df, new_row_df])


In [109]:
new_df.to_csv('../output/foster_cats_test_new.csv', index=False)

### Compare Files

In [73]:
old_df = pd.read_csv('../output/foster_cats_test_old.csv')
new_df = pd.read_csv('../output/foster_cats_test_new.csv')

In [160]:
# merge dfs and compare
compare_df = pd.merge(old_df, new_df, how='outer', on='cat_id', suffixes=['_old','_new'])
loop_cols = [t for t in old_df.columns if 'cat_id' not in t and 'import_at' not in t]

for col in loop_cols:
    col_compare_df = compare_df[[c for c in compare_df.columns if col in c]]
    compare_df[f'was_{col}_modified'] = (col_compare_df[f'{col}_old'] != col_compare_df[f'{col}_new'])

# discard rows where nothing was changed
compare_df['was_modified'] = compare_df[[col for col in compare_df.columns \
                                        if 'was_' in col]].any(axis=1)
compare_df = compare_df[compare_df['was_modified'] == True]

# distinguish adds, deletions, modifications
def determine_change(old_val, new_val):
    if pd.isnull(old_val):
        return 'row_added'
    elif pd.isnull(new_val):
        return 'row_deleted'
    else:
        return 'row_modified'

compare_df['change_type'] = compare_df.apply(lambda x: determine_change(x['cat_name_old'], x['cat_name_new']), 
                                             axis=1)

In [179]:
change_df = pd.read_csv('../output/foster_change_log.csv')

In [183]:
if change_df[change_df['change_type'] == 'row_added'].shape[0] > 0:
    print('new cats added!')

new cats added!


In [194]:
send_email(df, alert_message='change log test email', sender=my_username, password=my_password)

In [204]:
from foster_cats import *

In [221]:
# get last run + current run
old_fosters_df = pd.read_csv('../src/data/latest_fosters.csv')
new_fosters_df = getFosterInfo()

# split out html info from current run to avoid clutter in .csv
# we only need html for email alerts we might send 
new_fosters_write_df = new_fosters_df.drop(columns=['full_html'])

# save current run as history for next comparison
new_fosters_write_df.to_csv('../src/data/latest_fosters.csv', index=False)

# compare new and old runs to determine changes
changes_df = compareSnapshots(old_fosters_df, new_fosters_write_df)


if changes_df.shape[0] > 0:
    # logging.info('there have been changes since the last pipeline run')
    # logging.info('summary of changes: ')
    # logging.info(changes_df['change_type'].value_counts())
    
    # append changes to change history if it exists
    path = '../output/foster_change_log.csv'
    if os.path.exists(path):
        old_changes_df = pd.read_csv(path)
        changes_df = pd.concat([old_changes_df, changes_df])
    changes_df.to_csv(path, index=False)
    
    # if there were rows added, trigger alert

else:
    # logging.info('no changes since previous pipeline run')
    print('hi')

hi


In [224]:
import smtplib
from email.mime.text import MIMEText
from configparser import ConfigParser

parser = ConfigParser()
_ = parser.read('../credentials.cfg')
my_username = parser.get('my_email','my_username')
my_password = parser.get('my_email','my_password')


sender_email = my_username
sender_password = my_password
recipient_email = my_username
subject = f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M')}: New foster cat alert! {new_fosters_df.loc[0]['cat_summary']}"
body = f'''
    <html>
        <body>
        <h1> New cat alert! </h1>
            {new_fosters_df.loc[0]['full_html']}
        </body>
    </html>
    '''
html_message = MIMEText(body, 'html')
html_message['Subject'] = subject
html_message['From'] = sender_email
html_message['To'] = recipient_email

server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
server.login(sender_email, sender_password)
server.sendmail(sender_email, recipient_email, html_message.as_string())
server.quit()

(221,
 b'2.0.0 closing connection y12-20020ac8524c000000b003b643951117sm2170934qtn.38 - gsmtp')

In [199]:
cats[0]

<div class="comp-kjevao4x5 YzqVVZ" id="comp-kjevao4x5__61864364-ded1-4b6b-9703-ce7cfb79533b"><div class="MW5IWV" data-hook="bgLayers" id="bgLayers_comp-kjevao4x5"><div class="LWbAav Kv1aVt" data-testid="colorUnderlay"></div><div class="VgO9Yg" id="bgMedia_comp-kjevao4x5"></div></div><div class="" data-mesh-id="comp-kjevao4x5__61864364-ded1-4b6b-9703-ce7cfb79533binlineContent" data-testid="inline-content"><div data-mesh-id="comp-kjevao4x5__61864364-ded1-4b6b-9703-ce7cfb79533binlineContent-gridContainer" data-testid="mesh-container-content"><div class="comp-kjevao535 MazNVa rYiAuL" id="comp-kjevao535__61864364-ded1-4b6b-9703-ce7cfb79533b" title="
This sweet mom is so ready for retirement! Living in an Astoria backyard she had at least three litters over the past couple years. She's finally fixed and ready to move on to the good life! Perhaps a bit shy at first but a big fan of pets.

** Select Foster Pick Up (Cafe) when scheduling****"><a class="j7pOnl" data-testid="linkElement" href="ht

In [197]:
from configparser import ConfigParser
import ssl

parser = ConfigParser()
_ = parser.read('../credentials.cfg')
my_username = parser.get('my_email','my_username')
my_password = parser.get('my_email','my_password')

# Define the HTML document
body = '''
    <html>
        <body>
            <h1>Daily S&P 500 prices report</h1>
            <p>Hello, welcome to your report!</p>
        </body>
    </html>
    '''

# Generate today's date to be included in the email Subject
date_str = str(datetime.datetime.now())

# Create a MIMEMultipart class, and set up the From, To, Subject fields
email_message = MIMEMultipart()
email_message['From'] = my_username
email_message['To'] = my_username
email_message['Subject'] = f'Report email - {date_str}'

# Attach the html doc defined earlier, as a MIMEText html content type to the MIME message
email_message.attach(MIMEText(html, "html"))
# Convert it as a string
email_string = email_message.as_string()

# Connect to the Gmail SMTP server and Send Email
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.login(my_username, my_password)
    server.sendmail(my_username, my_password, email_string)

AttributeError: module 'datetime' has no attribute 'now'

# Appendix (Scratch)

In [ ]:
### first go to individual cat page and you can get the attributes via the below:

cafe_prospect = cat_soup.find('div', class_='comp-kjevammt2 YzqVVZ')


health_concerns = cat_soup.find('div', class_='comp-kjevamml YzqVVZ')


upcoming_appointments = cat_soup.find('div', class_='comp-kjevammp3 YzqVVZ')

